# 🎙️ Audio to Text with OpenAI Whisper

This notebook uses [OpenAI Whisper](https://github.com/openai/whisper) to transcribe audio files to text.

**Supported languages:** English 🇺🇸 and Spanish 🇪🇸

## Setup

1. Go to **Runtime → Change runtime type** and select **GPU** (T4 recommended) for faster transcription.
2. Run the cells below in order.

## 1. Install Dependencies

In [ ]:
!pip install -q openai-whisper

## 2. Import Libraries

In [ ]:
import whisper
import torch
from google.colab import files

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 3. Load the Whisper Model

Available model sizes (larger = more accurate but slower):

| Model  | Parameters | Relative Speed | Notes |
|--------|-----------|----------------|-------|
| tiny   | 39M       | ~32x           | Fastest, least accurate |
| base   | 74M       | ~16x           | Good balance for short clips |
| small  | 244M      | ~6x            | Good accuracy |
| medium | 769M      | ~2x            | Great for Spanish |
| large  | 1550M     | 1x             | Best accuracy, slowest |

In [ ]:
#@title Select Model Size { run: "auto" }
model_size = "medium" #@param ["tiny", "base", "small", "medium", "large"]

print(f"Loading Whisper '{model_size}' model...")
model = whisper.load_model(model_size, device=device)
print(f"Model '{model_size}' loaded successfully on {device}.")

## 4. Upload Your Audio File

Supported formats: `.mp3`, `.wav`, `.m4a`, `.flac`, `.ogg`, `.webm`, and more.

In [ ]:
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]
print(f"Uploaded: {audio_file}")

## 5. Transcribe Audio

Choose the language of your audio:
- `English` — transcribe English audio
- `Spanish` — transcribe Spanish audio
- `Auto-detect` — let Whisper detect the language automatically

In [ ]:
#@title Transcription Settings { run: "auto" }
language = "Auto-detect" #@param ["Auto-detect", "English", "Spanish"]

language_map = {
    "English": "en",
    "Spanish": "es",
    "Auto-detect": None
}
lang_code = language_map[language]

print(f"Transcribing '{audio_file}' (language: {language})...")
result = model.transcribe(audio_file, language=lang_code)

detected_lang = result.get("language", "unknown")
print(f"Detected language: {detected_lang}")
print("\n" + "=" * 60)
print("TRANSCRIPTION")
print("=" * 60)
print(result["text"])

## 6. View Timestamped Segments

Each segment includes start/end times and the corresponding text.

In [ ]:
print(f"{'Start':>8}  {'End':>8}  Text")
print("-" * 60)
for seg in result["segments"]:
    start = f"{seg['start']:.1f}s"
    end = f"{seg['end']:.1f}s"
    print(f"{start:>8}  {end:>8}  {seg['text'].strip()}")

## 7. Save Transcription to File

In [ ]:
import os

base_name = os.path.splitext(audio_file)[0]
output_file = f"{base_name}_transcription.txt"

with open(output_file, "w", encoding="utf-8") as f:
    f.write(f"Audio File: {audio_file}\n")
    f.write(f"Language: {detected_lang}\n")
    f.write(f"Model: {model_size}\n")
    f.write("=" * 60 + "\n\n")
    f.write("FULL TRANSCRIPTION:\n")
    f.write(result["text"] + "\n\n")
    f.write("TIMESTAMPED SEGMENTS:\n")
    for seg in result["segments"]:
        f.write(f"[{seg['start']:.1f}s - {seg['end']:.1f}s] {seg['text'].strip()}\n")

print(f"Transcription saved to: {output_file}")
files.download(output_file)

## 8. (Optional) Translate Spanish Audio to English

Whisper can also **translate** non-English audio directly to English text.

In [ ]:
#@title Translate to English { run: "auto" }
translate = False #@param {type:"boolean"}

if translate:
    print(f"Translating '{audio_file}' to English...")
    translation = model.transcribe(audio_file, task="translate")
    print("\n" + "=" * 60)
    print("ENGLISH TRANSLATION")
    print("=" * 60)
    print(translation["text"])
else:
    print("Set 'translate' to True above and re-run this cell to translate.")